# Sistema de Recomendação de Livros

**Nome do Arquivo:** tratamento_dados.ipynb  
**Autor:** Marcos Paulo Cyrillo da Silva  
**Data de Criação:** 30/09/2025  
**Última Modificação:** 05/10/2025  
**Descrição:** Este arquivo corresponde a um projeto de recomendação de livros a partir da leitura atual do usuário. Nele, busca-se realizar a recomendação utilizando um modelo proveniente do modelo de aprendizado de máquina K-Vizinhos Mais Próximos (KNN). Para isto, nele estão incluídas as etapas obtenção de datasets, tratamento de dados, treinamento do modelo e uma visualização simplificada do resultado: lista dos livros recomendados. 

# 1. Obtenção e tratamento dos dados

## 1.1 Obtenção dos dados
Os dados utilizados no treinamento do modelo KNN para a recomendação dos livros foram retirados dos datasets obtidos mediante plataforma Kaggle. Para realizar a implementação do sistema utilizou-se dois datasets, um referente aos livros e suas informações correspondentes ('books.csv') e o outro referente às avaliações dos usuários para esses livros ('ratings.csv'). O link para consulta dos datasets é apresentado a seguir.

**Link:** https://www.kaggle.com/datasets/zygmunt/goodbooks-10k

## 1.2 Tratamento dos dados
O tratamento dos dados seguiu as etapas de visualização inicial, para compreender como eles estavam organizados. Em seguida, realizou-se a etapa de tratamento e limpeza, com a finalidade de filtrar apenas as informações relevantes para o presente projeto. Por fim, foram realizadas as etapas de transformação dos dados para seu emprego no modelo KNN.

### 1.2.1 Importando as bibliotecas e pacotes

In [1]:
import pandas as pd
from pathlib import Path
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import joblib

Sobre as bibliotecas e pacotes utilizados:
* **pandas:** utilizado para a manipulação e análise dos dados dos datasets.
* **pathlib:** utilizado para o gerenciamento dos caminhos dos arquivos dos datasets e dos arquivos de saída.
* **csr_matrix:** utilizado para a criação de uma matriz esparsa após o tratamento dos dados.
* **NearestNeighbors:** utilizado para o treinamento do modelo KNN.
* **joblib:** utilizado para o salvamento do modelo treinado e da matriz pivot tratada, possibilitando o uso dessas informações em outros ambientes de desenvolvimento.

### 1.2.2 Definindo os caminhos para a raiz do projeto e para os datasets

In [2]:
ROOT_DIR = Path().resolve().parent # Define a raiz do projeto
RAW_DIR = ROOT_DIR / "data" / "raw" # Define o caminho para os datasets

### 1.2.3 Carregando os dataframes com pandas ('read_csv()')

In [3]:
livros_df = pd.read_csv(RAW_DIR / "books.csv") # Carrega o dataframe com os dados dos livros
livros_df.head() # Exibe o cabeçalho e as primeiras 5 linhas do dataframe

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
avaliacoes_df = pd.read_csv(RAW_DIR / "ratings.csv") # Carregando o dataframe com os dados das avaliações dos usuários
avaliacoes_df.head() # Exibe o cabeçalho e as primeiras 5 linhas do dataframe

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


### 1.2.4 Exibindo todas as colunas dos dataframes

In [5]:
livros_df.columns

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')

In [6]:
avaliacoes_df.columns

Index(['book_id', 'user_id', 'rating'], dtype='object')

Para o dataframe dos livros ('livros_df'), apenas as colunas 'id', 'original_title' e 'ratings_count' serão utilizadas.
* **id:** identificador único de cada livro no dataset.
* **original_title:** título original da obra, antes de traduções ou adaptações.
* **ratings_count:** quantidade total de avaliações recebidas pelo livro.

Para o dataframe das avaliações ('avaliacoes_df'), todas as colunas serão utilizadas.
* **book_id:** identificador do livro avaliado.
* **user_id:** identificador do usuário que realizou a avaliação.
* **rating:** nota atribuída pelo usuário ao livro.

### 1.2.5 Selecionando apenas as colunas de interesse do dataframe 'livros_df'

In [7]:
livros_df = livros_df[["id", "original_title", "ratings_count"]] # Seleciona apenas as colunas 'id', 'original_title' e 'ratings_count'
livros_df.head() # Exibe as primeiras linhas com as colunas selecionadas

,id,original_title,ratings_count
0,1,The Hunger Games,4780653
1,2,Harry Potter and the Philosopher's Stone,4602479
2,3,Twilight,3866839
3,4,To Kill a Mockingbird,3198671
4,5,The Great Gatsby,2683664


### 1.2.6 Renomeando as colunas dos dataframes
Para melhorar a identificação de cada coluna dos datrafames foi realizada a renomeação das colunas. Os nomes originais e os novos nomes atribuídos são mostrados a seguir.

**Dataframe 'livros_df':**
* **id** -> ID_LIVRO
* **original_title** -> TITULO_LIVRO
* **ratings_count** -> QTD_AVALIACOES
  
**Dataframe 'avaliacoes_df':**
* **book_id** -> ID_LIVRO
* **user_id** -> ID_USUARIO
* **rating** -> AVALIACAO

In [8]:
livros_df = livros_df.rename(columns={"id": "ID_LIVRO", "original_title":"TITULO_LIVRO", "ratings_count": "QTD_AVALIACOES"}) # Renomeia as colunas utilizando o método 'rename()'
livros_df.head() # Exibe as primeiras 5 linhas do dataframe após a renomeação das colunas

,ID_LIVRO,TITULO_LIVRO,QTD_AVALIACOES
0,1,The Hunger Games,4780653
1,2,Harry Potter and the Philosopher's Stone,4602479
2,3,Twilight,3866839
3,4,To Kill a Mockingbird,3198671
4,5,The Great Gatsby,2683664


In [9]:
avaliacoes_df = avaliacoes_df.rename(columns={"book_id": "ID_LIVRO", "user_id": "ID_USUARIO", "rating": "AVALIACAO"}) # Renomeia as colunas utilizando o método 'rename()'
avaliacoes_df.head() # Exibe as primeiras 5 linhas do dataframe após a renomeação das colunas

,ID_LIVRO,ID_USUARIO,AVALIACAO
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


### 1.2.7 Verificando se há valores nulos no dataset

In [10]:
livros_df.isna().sum() # Verifica se há valores nulos no dataset, retornando a quantidade desses valores se existirem

ID_LIVRO            0
TITULO_LIVRO      585
QTD_AVALIACOES      0
dtype: int64

Como há valores nulos encontrados em 'TITULO_LIVRO', é realizada a exclusão desses valores.

In [11]:
livros_df = livros_df.dropna() # Exclui as linha que apresentam valores nulos

livros_df.isna().sum() # Faz uma nova verificação de confirmação

ID_LIVRO          0
TITULO_LIVRO      0
QTD_AVALIACOES    0
dtype: int64

In [12]:
avaliacoes_df.isna().sum() # Verifica se há valores nulos no dataset, retornando a quantidade desses valores se existirem

ID_LIVRO      0
ID_USUARIO    0
AVALIACAO     0
dtype: int64

No dataset 'avalaiacoes_df' não foram encontrados valores nulo, então o processo de exclusão não precisa ser realizado.

### 1.2.8 Aplicando regras de negócio
Para que o modelo KNN seja treinado de maneira eficiente, é necessário considerar algumas regras de negócio. Assim, são incluídos apenas os livros que possuem mais de 999 avaliações e os usuários que realizaram mais de 99 avaliações. Essa filtragem é fundamental, pois reduz relações fracas entre usuários e livros, melhorando a qualidade dos vizinhos identificados pelo modelo e, consequentemente, a precisão das recomendações.

In [13]:
livros_df.shape # Verifica as dimensões do dataset

(9415, 3)

In [14]:
livros_df = livros_df[livros_df["QTD_AVALIACOES"] > 999] # Filtra o dataset e seleciona apenas as linhas que possuem 'QTD_AVALIACOES' maior que 999
livros_df.shape # Verifica novamente as dimensões do dataset para confirmar a seleção das linhas

(9415, 3)

In [15]:
avaliacoes_df.shape # Verifica as dimensões do dataset

(981756, 3)

In [16]:
avaliacoes_df["ID_USUARIO"].value_counts() # Verifica o maior número de avaliações

ID_USUARIO
12874    200
30944    200
28158    199
52036    199
12381    199
        ... 
47248      2
48186      2
17867      2
11638      2
43877      2
Name: count, Length: 53424, dtype: int64

In [17]:
qtd_avaliacoes = avaliacoes_df["ID_USUARIO"].value_counts() > 99 # Filtra os usuários que avaliaram mais de 99 vezes
lista_indices = qtd_avaliacoes[qtd_avaliacoes].index # # Obtém o índice dos usuários que avaliaram mais de 99 vezes
lista_indices.shape # Verifica as dimensões da lista

(1199,)

In [18]:
avaliacoes_df.shape # Verifica as dimensões do dataframe

(981756, 3)

In [19]:
avaliacoes_df = avaliacoes_df[avaliacoes_df["ID_USUARIO"].isin(lista_indices)] # Considera apenas os usuários do dataframe 'avaliacoes_df' que estão na lista 'lista_indices'
avaliacoes_df.shape # Verifica as dimensões do dataframe

(165905, 3)

### 1.2.9 Unindo os dataframes
Para a finalização da etapa de tratamento de dados, serão realizadas a união dos dataframes e a verificação de linhas repetidas após a união. Para realizar a união, é necessário primeiro verificar se a coluna de referência, aquela presente em ambos os dataframes e que representa a mesma informação, possui o mesmo tipo em cada dataframe.

In [20]:
livros_df.info() # Mostra informações sobre o dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 9415 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   ID_LIVRO        9415 non-null   int64 
 1   TITULO_LIVRO    9415 non-null   object
 2   QTD_AVALIACOES  9415 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 294.2+ KB


In [21]:
avaliacoes_df.info() # Mostra informações sobre o dataframe

<class 'pandas.core.frame.DataFrame'>
Index: 165905 entries, 0 to 981737
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype
---  ------      --------------   -----
 0   ID_LIVRO    165905 non-null  int64
 1   ID_USUARIO  165905 non-null  int64
 2   AVALIACAO   165905 non-null  int64
dtypes: int64(3)
memory usage: 5.1 MB


A coluna utilizada para realizar a união dos dataframes é a coluna 'ID_LIVRO'. Como em ambos os dataframes ela possui o mesmo tipo (int64), a união pode ser realizada.

In [22]:
livros_avaliacoes_df = avaliacoes_df.merge(livros_df, on="ID_LIVRO") # Une os dois dataframes considerando a coluna 'ID_LIVRO' como referência
livros_avaliacoes_df.head() # Verifica as primeiras 5 linhas do dataframe

,ID_LIVRO,ID_USUARIO,AVALIACAO,TITULO_LIVRO,QTD_AVALIACOES
0,1,314,5,The Hunger Games,4780653
1,1,439,3,The Hunger Games,4780653
2,1,588,5,The Hunger Games,4780653
3,1,1169,4,The Hunger Games,4780653
4,1,1185,4,The Hunger Games,4780653


Por fim, é realizada a verificação de conteúdos duplicados no dataframe, caso exista eles são excluídos.

In [23]:
livros_avaliacoes_df.shape # Verifica as dimensões do dataframe

(161857, 5)

In [24]:
livros_avaliacoes_df = livros_avaliacoes_df.drop_duplicates(["ID_USUARIO", "ID_LIVRO"]) # Verifica se há linhas duplicadas no dataframe e as remove
livros_avaliacoes_df.shape # Verifica as dimensões do dataframe

(161626, 5)

Ao final da etapa de tratamento de dados, obtém-se o dataframe ('livros_avaliacoes_df') que será utilizado para o treinamento do modelo KNN.

## 2. Treinamento do modelo KNN
Para aplicar o modelo KNN utilizando o dataframe obtido nas etapas anteriores, os dados de avaliações devem ser organizados em uma tabela pivô, onde linhas representam livros, colunas representam usuários e células contêm as avaliações. Além disso, os valores ausentes da tabela pivô precisam ser preenchidos com 0 para garantir consistência na matriz. Em seguida, a tabela deve ser convertida em uma matriz esparsa, o que economizará memória e reduzirá o custo de processamento.

### 2.1 Criação da tabela pivô

In [25]:
# Cria uma tabela pivô a partir do do dataframe 'livros_avaliacoes_df'
# Cada linha representa um livro ('TITULO_LIVRO')
# Cada coluna representa um usuário ('ID_USUARIO')
# Os valores na tabela são as avaliações dadas pelos usuários ('AVALIACAO')
livros_pivot = livros_avaliacoes_df.pivot_table(columns="ID_USUARIO", index="TITULO_LIVRO", values="AVALIACAO")
livros_pivot.head() # Verifica as primeiras 5 linhas do dataframe

ID_USUARIO,35,173,178,274,314,368,439,588,589,725,...,52929,52956,52965,52994,53145,53173,53245,53292,53293,53366
TITULO_LIVRO,,,,,,,,,,,,,,,,,,,,,
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A Monster Calls,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Animal Farm & 1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
"Burned (Burned, #1)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Call the midwife : a true story of the East End in the 1950s,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
livros_pivot = livros_pivot.fillna(0) # Preenche os valores nulos do dataframe com o valor 0
livros_pivot.head() # Verifica novamente as primeiras 5 linhas do dataframe

ID_USUARIO,35,173,178,274,314,368,439,588,589,725,...,52929,52956,52965,52994,53145,53173,53245,53292,53293,53366
TITULO_LIVRO,,,,,,,,,,,,,,,,,,,,,
,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Monster Calls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Animal Farm & 1984,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
"Burned (Burned, #1)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Call the midwife : a true story of the East End in the 1950s,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2.2 Convertendo a tabela pivô em uma matriz esparsa

In [27]:
livros_sparse = csr_matrix(livros_pivot) # Converte a matriz 'livros_pivot' em uma matriz esparsa
type(livros_sparse) # Verifica o tipo do objeto criado

scipy.sparse._csr.csr_matrix

### 2.3 Treinando o modelo KNN
O modelo KNN é um algoritmo de aprendizado baseado em instâncias que funciona identificando os K vizinhos mais próximos de um ponto de referência em um espaço de características. Para isso, ele calcula a distância entre o ponto de interesse e todos os outros pontos do conjunto de dados, selecionando os mais próximos como vizinhos. A partir desses vizinhos, é possível realizar tarefas como classificação (votação majoritária) ou regressão (média dos valores).

O KNN não aprende parâmetros ou pesos, como modelos paramétricos; seu “treinamento” consiste apenas em armazenar os dados de referência para que consultas futuras possam ser realizadas rapidamente.

No scikit-learn, o KNN é implementado pela classe NearestNeighbors, que permite escolher o algoritmo de busca (por exemplo, brute para força bruta) e a métrica de distância (como cosine, euclidean, etc.), dependendo do tipo de dados e da aplicação. No presente projeto são utilizados os seguintes parâmetros:

* **algorithm="brute":** força bruta, que calcula todas as distâncias entre os livros; escolhido por ser eficiente para matrizes esparsas, como a tabela pivô obtida.
* **metric="cosine":** distância cosseno, que neste projeto mede a semelhança entre livros com base no padrão de avaliações dos usuários, ignorando diferenças absolutas; adequada para capturar preferências semelhantes.

In [28]:
modelo = NearestNeighbors(algorithm="brute", metric="cosine") # Cria o modelo KNN utilizando o algoritmo de força bruta ('brute') e a métrica de similaridade 'cosine'
modelo.fit(livros_sparse) # Treina o modelo KNN com a matriz esparsa de livros

NearestNeighbors(algorithm='brute', metric='cosine')

Após o treinamento do modelo KNN, obtém-se uma estrutura capaz de identificar os itens mais semelhantes a qualquer ponto de referência. Isso permite encontrar os vizinhos mais próximos de um dado índice (título do livro), ou seja, os livros que possuem padrões de avaliação semelhantes por parte dos usuários. Esses vizinhos representam os livros que têm maior afinidade em termos de preferências dos usuários e, com isso, podem ser apresentados como recomendações personalizadas.

A seguir são mostrados dois exemplos para os livros de títulos **"The Hunger Games"** e **Twilight**.

In [29]:
titulo_livro = "The Hunger Games"  # Define o título do livro de referência para gerar recomendações
try:
    index = livros_pivot.index.get_loc(titulo_livro)  # Tenta obter o índice do livro na matriz 'livros_pivot'
except KeyError:
    print("Livro não encontrado na base de dados!")  # Caso o título não exista, exibe uma mensagem de erro
    index = None  # Define 'index' como None para evitar novos erros
if index is not None:
    distances, suggestions = modelo.kneighbors(livros_pivot.values[index].reshape(1, -1)) # Encontra os vizinhos mais próximos ao livro de referência
    print(f"Recomendações para {titulo_livro}:")  # Exibe o título do livro de referência
    for i in suggestions[0]:  # Percorre os índices dos livros recomendados exibindo os títulos recomendados
        print(livros_pivot.index[i])  

Recomendações para The Hunger Games:
The Hunger Games
Catching Fire
The Help
Harry Potter and the Philosopher's Stone
Mockingjay


Para o livro **"The Hunger Games"** as recomendações obtidas foram:
* The Hunger Games
* Catching Fire
* The Help
* Harry Potter and the Philosopher's Stone
* Mockingjay

In [30]:
titulo_livro = "Twilight"  # Define o título do livro de referência para gerar recomendações
try:
    index = livros_pivot.index.get_loc(titulo_livro)  # Tenta obter o índice do livro na matriz 'livros_pivot'
except KeyError:
    print("Livro não encontrado na base de dados!")  # Caso o título não exista, exibe uma mensagem de erro
    index = None  # Define 'index' como None para evitar novos erros
if index is not None:
    distances, suggestions = modelo.kneighbors(livros_pivot.values[index].reshape(1, -1)) # Encontra os vizinhos mais próximos ao livro de referência
    print(f"Recomendações para {titulo_livro}:")  # Exibe o título do livro de referência
    for i in suggestions[0]:  # Percorre os índices dos livros recomendados exibindo os títulos recomendados
        print(livros_pivot.index[i])  

Recomendações para Twilight:
Twilight
The Hunger Games
Harry Potter and the Philosopher's Stone
Memoirs of a Geisha
Pride and Prejudice


Para o livro **"Twilight"** as recomendações obtidas foram:

* Twilight
* The Hunger Games
* Harry Potter and the Philosopher's Stone
* Memoirs of a Geisha
* Pride and Prejudice

### 2.4 Exportando o modelo e a tabela pivô

In [31]:
PROCESSED_DIR = ROOT_DIR / "data" / "processed"  # Define o diretório onde os arquivos processados serão salvos

joblib.dump(modelo, PROCESSED_DIR / "modelo_knn.pkl") # Salva o modelo KNN treinado em formato .pkl para ser utilizado posteriormente
joblib.dump(livros_pivot, PROCESSED_DIR / "livros_pivot.pkl") # Salva a tabela 'livros_pivot' tratada, possibilitando reutilização em outros ambientes

['C:\\Users\\marco\\OneDrive\\nuvem\\projeto_recomendador_livros\\data\\processed\\livros_pivot.pkl']

## 3 Conclusão 
As recomendações obtidas para os livros citados são coerentes e apresentam semelhança entre os títulos sugeridos. Isso indica que o treinamento do modelo foi realizado corretamente, cumprindo o objetivo de fornecer recomendações de livros de forma adequada. É importante ressaltar que melhorias na implementação podem e devem ser realizadas, uma vez que a presente versão tem como foco o estudo das técnicas empregadas, e não a otimização da abordagem em si.